
#### Write a Bash script to reconstruct the Shopping Bill dataset from the OCR data files and store it in a separate file for each card.
#### The reconstructed dataset should contain one file for each card with the name 
    
    shopping_bill_<card_number>.txt,  "<card_number>" is a number of the card e.g. '1', '4', '12', '24' etc.

#### The generated files should contain the data in the format given below.


```
        SHOPNAME:CUSTOMER_NAME:CARD_NO
        Item:Category:Qty:Price:Cost
        Carrots:Vegetable/Food:1.5:50:75
        ....
        ....
```

#### Where information like shopname, category, item. cost etc. can be fetched from their respective 'ocr_.txt' files.<br> 
#### Or you can find all the information in a single file named 'ocr_full.txt'<br>
#### An example output for the output file shopping_bill_1.txt is given below.


        SV Stores:Srivatsan:1
        Item:Category:Qty:Price:Cost
        Carrots:Vegetables/Food:1.5:50:75
        Soap:Toiletries:4:32:128
        Tomatoes:Vegetables/Food:2:40:80
        Bananas:Vegetables/Food:8:8:64
        Socks:Footwear/Apparel:3:56:168
        Curd:Dairy/Food:0.5:32:16
        Milk:Dairy/Food:1.5:24:36



__The description of the contents in the data files is given in the table below,__

| Filename |	Information stored | 
| --- | --- |
|[__ocr_categories.txt__](Data/ocr_categories.txt)|__category values of each card__|
|[__ocr_head.txt__](Data/ocr_head.txt)|__store name, customer name and card no of each card__|
|[__ocr_names.txt__](Data/ocr_names.txt)|__customer name of each card__|
|[__ocr_cost.txt__](Data/ocr_cost.txt)|__cost column of each card__|
|[__ocr_item_record.txt__](Data/ocr_item_record.txt)|__items purchased with price, quantity and cost__|
|[__ocr_seqno.txt__](Data/ocr_seqno.txt)|__card number of each card__|
|[__ocr_full.txt__](Data/ocr_full.txt)|__full information of each card__|
|[__ocr_items.txt__](Data/ocr_items.txt)|__items column of each card__|
|[__ocr_shopname.txt__](Data/ocr_shopname.txt)|__shop name of each card__|


utputAll the above files are located in the current working directory.

#### Note:
 - The data('ocr_' files) was created from the images(available in the data) using some optical character reader utility
 - hence these files may contain some incorrectly read data,  
     - e.g some character/digits/words may be misread as some other character or words. So your script may need to do some data correction as well. 
 - To check these you can use the corrected and generated output files that are uploaded here(generated files). 
     - You can download these and match with your generated files on your local machine.
 - Try to make use of grep/sed/awk inside your Bash scripts wherever required.

#### Names of Customers

 - Srivatsan
 - Sudeep
 - Akshaya
 - Advaith
 - Akshaya
 - Srivatsan
 - Akhil
 - Advaith
 - Mohith
 - Rajesh
 - Abhinav
 - Aparna
 - Abhinav
 - Vignesh
 - Abhinav
 - Advaith
 - George
 - Ahmed
 - Neeraja
 - Ahmed
 - Akshaya
 - Suresh
 - Julia
 - Neeraja
 - Ahmed
 - Srivat - san
 - Srivatsan
 - Vignesh
 - Ahmed
 - Radha
 
#### Names of Stores

 - SV Stores
 - Big Bazaar
 - SV Stores
 - SV Stores
 - Big Bazaar
 - Sun General
 - SV Stores
 - Big Bazaar
 - Big Bazaar
 - Sun General
 - SV Stores
 - Sun General
 - Big Bazaar
 - Sun General
 - Sun General
 - Sun General
 - SV Stores
 - SV Stores
 - SV Stores
 - SV Stores
 - SV Stores
 - Sun General
 - SV Stores
 - SV Stores
 - Sun General
 - SV Stores
 - Sun General
 - SV Stores
 - SV Stores
 - Big Bazaar

In [196]:
shop_names=()
names=()
my_script(){

idx=0
while read line; do
    brac='^\['
    if ! [[ $line =~ $brac ]]; then
        shop_names[$idx]=$line
        # echo $line
        idx=$idx+1
    fi
done < Data/ocr_shopname.txt

# echo ${shop_names[@]}

name_str=$(grep -v '^\[.*\]$' Data/ocr_names.txt | grep -o '[[:alpha:]]*$' | sed -e 's/Royal/Rajesh/g' -e 's/elite/Julia/g')
names=( $name_str )

# echo ${names[@]}


awk 'BEGIN{FS="\n"}{if (length($1) > 7){print $1}}' Data/ocr_item_record.txt | sed  's/^\[.*\]$//g' > records
N=30
tc=230
junk='^[SV]+[[:space:]]+[Stores]+'
junk1='^[Big]+[[:space:]]+[Bazaar]+'
junk2='^[Item]+[[:space:]]+[Category]+'
junk3='^[Sun]+[[:space:]]+[General]+'
first=0
prev="s"
for (( i=1;i<=$tc;i++ )){
    line=$(sed "${i}q;d" records)
    line_chars=${#line}
    if [[ ( $line =~ $junk ) || ( $line =~ $junk2 ) || ( $line =~ $junk1 ) || ( $line =~ $junk3 ) ]]; then
        continue
    fi
    if [[ $line != $prev ]]; then
        prev=$line;
        echo $line
    fi
} > nrec

awk 'BEGIN{FS="\n"}{if (length($1) == 0){print "0 0 0 0 0"}else{print $1}}' nrec | head -n -1 > nrec2
awk '
BEGIN{
    FS=" "
    item_name=""
    item_categ=""
    item_quant=""
    item_price=""
    item_cost=""
}
{
if (NF==6){
    item_name=$1" "$2
    item_categ=$3
    item_quant=$4
    item_price=$5
    item_cost=$6
}
else{
    item_name=$1
    item_categ=$2
    item_quant=$3
    item_price=$4
    item_cost=$5
}
print item_name":"item_categ":"item_quant":"item_price":"item_cost
}
' nrec2 > flist
idx=1
TC=185
empty='^0'
for (( i=1;i<=185;i++ )) {  
    line=$(sed "${i}q;d" flist)
    idx_=$(( $idx - 1 ))
    if [[ $line =~ $empty ]]; then
        touch myout/shopping_bill_$idx.txt
        echo ${shop_names[$idx_]}:${names[$idx_]}:$idx >> myout/shopping_bill_$idx.txt
        echo Item:Category:Qty:Price:Cost >> myout/shopping_bill_$idx.txt
        idx=$(( idx +1 ))
        continue
    fi
    curr_idx=$(( idx -1 ))
    echo $line >> myout/shopping_bill_$curr_idx.txt
    
}
### Final Edits
sed -e 's/1\:a2/1\:32/g' -e 's/a\:24\:72/3\:24\:72/g' myout/shopping_bill_26.txt > chrck
cat chrck > myout/shopping_bill_26.txt
sed -e 's/\=Canned/Canned/g' myout/shopping_bill_5.txt > chrck
cat chrck > myout/shopping_bill_5.txt

rm flist nrec nrec2 records chrck
}
my_script

# echo ${sb[1]}

In [202]:
sed -e 's/\=Canned/Canned/g' myout/shopping_bill_5.txt > chrck
cat chrck > myout/shopping_bill_5.txt

In [182]:
idx=1
TC=185
empty='^0'
for (( i=1;i<=185;i++ )) {  
    line=$(sed "${i}q;d" flist)
    idx_=$(( $idx - 1 ))
    if [[ $line =~ $empty ]]; then
        touch myout/shopping_bill_$idx.txt
        echo ${shop_names[$idx_]}:${names[$idx_]}:$idx >> myout/shopping_bill_$idx.txt
        echo Item:Category:Qty:Price:Cost >> myout/shopping_bill_$idx.txt
        idx=$(( idx +1 ))
        continue
    fi
    curr_idx=$(( idx -1 ))
    echo $line >> myout/shopping_bill_$curr)idx.txt
    
}


In [160]:
rbl(){
    awk '
    BEGIN{
        idx=0;
    }
    {
        if ( $0 !~ /^\[/ ){
            val[idx]=$0
            idx=idx+1
        }
    }
    END{
    
           for (i in val){
           if (i == idx-1){
                 print val[i]
                 }
            else {
               print val[i]":"
                }
           }
           
    
    }
    ' $1

} 

^C


In [43]:
echo $(rbl Data/ocr_shopname.txt)

30 SV Stores: Big Bazaar: SV Stores: SV Stores: Big Bazaar: Sun General: SV Stores: Big Bazaar: Big Bazaar: Sun General: SV Stores: Sun General: Big Bazaar: Sun General: Sun General: Sun General: SV Stores: SV Stores: SV Stores: SV Stores: SV Stores: Sun General: SV Stores: SV Stores: Sun General: SV Stores: Sun General: SV Stores: SV Stores: Big Bazaar


In [ ]:
grep -v "^[[:space:]]" Data/ocr_items.txt | sed  '/^\[.*\]$/ d' 
# 

- begin at \[]
- discard Item

In [94]:
for (( NUM=1;NUM<186;NUM++ )){
    item=$(sed "${NUM}q;d" items)
    category=$(sed "${NUM}q;d" categories)
    echo "$item:$category"
} > item_category
